# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Dec 19 2022 12:47PM  253377        15        8922916   
1  Dec 19 2022 12:45PM  253376        10  Yusen-8922915   
2  Dec 19 2022 12:36PM  253375        19        8922316   
3  Dec 19 2022 12:36PM  253375        19        8922317   
4  Dec 19 2022 12:36PM  253375        19        8922318   
5  Dec 19 2022 12:36PM  253375        19        8922331   
6  Dec 19 2022 12:36PM  253375        19        8922332   
7  Dec 19 2022 12:36PM  253375        19        8922333   
8  Dec 19 2022 12:36PM  253375        19        8922334   
9  Dec 19 2022 12:36PM  253375        19        8922335   

                    Customer ShipmentStatus  
0  Tripoint Therapeutics LLC       Released  
1          Yusen – 3D Matrix       Released  
2             Innogenix, LLC       Released  
3             Innogenix, LLC       Released  
4             Innogenix, LLC       Released  
5             Innogenix, LLC       Released  
6             Innogenix, LLC       Released  
7             Innogenix, LLC       Released  
8             Innogenix, LLC       Released  
9             Innogenix, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  253373       Released          1
22  253374       Released          2
23  253375       Released         27
24  253376       Released          1
25  253377       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
253373                NaN        NaN       1.0
253374                NaN        NaN       2.0
253375                NaN        NaN      27.0
253376                NaN        NaN       1.0
253377                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253332                1.0        3.0       9.0
253340                0.0       18.0       1.0
253344                0.0        0.0       6.0
253347                0.0       40.0       0.0
253348                0.0        0.0      54.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
253332                  1          3         9
253340                  0         18         1
253344                  0          0         6
253347                  0         40         0
253348                  0          0        54

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               253332          1          3         9
1               253340          0         18         1
2               253344          0          0         6
3               253347          0         40         0
4               253348          0          0        54

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               253332         1         3        9
1               253340                  18        1
2               253344                            6
3               253347                  40         
4               253348                           54

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Dec 19 2022 12:47PM  253377        15     Tripoint Therapeutics LLC
1   Dec 19 2022 12:45PM  253376        10             Yusen – 3D Matrix
2   Dec 19 2022 12:36PM  253375        19                Innogenix, LLC
29  Dec 19 2022 12:17PM  253374        19       GUSA Granules USA, Inc.
31  Dec 19 2022 12:06PM  253373        22             NBTY Global, Inc.
32  Dec 19 2022 11:53AM  253371        10  Snap Medical Industries, LLC
34  Dec 19 2022 11:53AM  253370        10  Snap Medical Industries, LLC
38  Dec 19 2022 11:43AM  253368        10                       Bio-PRF
39  Dec 19 2022 11:37AM  253367        19       GUSA Granules USA, Inc.
40  Dec 19 2022 11:31AM  253366        10             ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Dec 19 2022 12:47PM  253377        15     Tripoint Therapeutics LLC   
1  Dec 19 2022 12:45PM  253376        10             Yusen – 3D Matrix   
2  Dec 19 2022 12:36PM  253375        19                Innogenix, LLC   
3  Dec 19 2022 12:17PM  253374        19       GUSA Granules USA, Inc.   
4  Dec 19 2022 12:06PM  253373        22             NBTY Global, Inc.   
5  Dec 19 2022 11:53AM  253371        10  Snap Medical Industries, LLC   
6  Dec 19 2022 11:53AM  253370        10  Snap Medical Industries, LLC   
7  Dec 19 2022 11:43AM  253368        10                       Bio-PRF   
8  Dec 19 2022 11:37AM  253367        19       GUSA Granules USA, Inc.   
9  Dec 19 2022 11:31AM  253366        10             ISDIN Corporation   

  Completed Executing Released  
0                            1  
1                            1  
2                           27  
3                            2  
4                            1  
5                            2  
6                            4  
7                            1  
8                            1  
9                            5

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Dec 19 2022 12:47PM  253377        15     Tripoint Therapeutics LLC   
1  Dec 19 2022 12:45PM  253376        10             Yusen – 3D Matrix   
2  Dec 19 2022 12:36PM  253375        19                Innogenix, LLC   
3  Dec 19 2022 12:17PM  253374        19       GUSA Granules USA, Inc.   
4  Dec 19 2022 12:06PM  253373        22             NBTY Global, Inc.   
5  Dec 19 2022 11:53AM  253371        10  Snap Medical Industries, LLC   
6  Dec 19 2022 11:53AM  253370        10  Snap Medical Industries, LLC   
7  Dec 19 2022 11:43AM  253368        10                       Bio-PRF   
8  Dec 19 2022 11:37AM  253367        19       GUSA Granules USA, Inc.   
9  Dec 19 2022 11:31AM  253366        10             ISDIN Corporation   

  Released Executing Completed  
0        1                      
1        1                      
2       27                      
3        2                      
4        1                      
5        2                      
6        4                      
7        1                      
8        1                      
9        5

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                   Customer Released  \
0  Dec 19 2022 12:47PM  253377        15  Tripoint Therapeutics LLC        1   
1  Dec 19 2022 12:45PM  253376        10          Yusen – 3D Matrix        1   
2  Dec 19 2022 12:36PM  253375        19             Innogenix, LLC       27   
3  Dec 19 2022 12:17PM  253374        19    GUSA Granules USA, Inc.        2   
4  Dec 19 2022 12:06PM  253373        22          NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                   Customer  Released  \
0  Dec 19 2022 12:47PM  253377        15  Tripoint Therapeutics LLC       1.0   
1  Dec 19 2022 12:45PM  253376        10          Yusen – 3D Matrix       1.0   
2  Dec 19 2022 12:36PM  253375        19             Innogenix, LLC      27.0   
3  Dec 19 2022 12:17PM  253374        19    GUSA Granules USA, Inc.       2.0   
4  Dec 19 2022 12:06PM  253373        22          NBTY Global, Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                   Customer  Released  \
0  Dec 19 2022 12:47PM  253377        15  Tripoint Therapeutics LLC       1.0   
1  Dec 19 2022 12:45PM  253376        10          Yusen – 3D Matrix       1.0   
2  Dec 19 2022 12:36PM  253375        19             Innogenix, LLC      27.0   
3  Dec 19 2022 12:17PM  253374        19    GUSA Granules USA, Inc.       2.0   
4  Dec 19 2022 12:06PM  253373        22          NBTY Global, Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3040298     172.0       58.0        0.0
15          760094      36.0       10.0        0.0
19         1266810      40.0        3.0        1.0
22          253373       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3040298     172.0       58.0        0.0
1        15   760094      36.0       10.0        0.0
2        19  1266810      40.0        3.0        1.0
3        22   253373       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     172.0       58.0        0.0
1        15      36.0       10.0        0.0
2        19      40.0        3.0        1.0
3        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released  172.0
1        15   Released   36.0
2        19   Released   40.0
3        22   Released    1.0
4        10  Executing   58.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    15    19   22
Status                           
Completed    0.0   0.0   1.0  0.0
Executing   58.0  10.0   3.0  0.0
Released   172.0  36.0  40.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    15    19   22
0          Completed    0.0   0.0   1.0  0.0
1          Executing   58.0  10.0   3.0  0.0
2           Released  172.0  36.0  40.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    15    19   22
0  Completed    0.0   0.0   1.0  0.0
1  Executing   58.0  10.0   3.0  0.0
2   Released  172.0  36.0  40.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()